# Milestone 1

In [11]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Download data and unzip the file

In [9]:
# Referenced the below code from lecture notes 

article_id = 14096681 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = os.path.join(os.path.pardir, "data/rainfall/")

In [10]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]            
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

## Combine CSVs into 1 (written to data/combined_data.csv)

In [19]:
%%time
files = glob.glob(os.path.join(os.path.pardir, "data/rainfall/*.csv"))
df = pd.concat((
  pd.read_csv(file, index_col=0)
  .assign(model=re.findall("/([^_]*)", file)[0])
  for file in files 
  if file != os.path.join(os.path.pardir, "data/rainfall/observed_daily_rainfall_SYD.csv")))
df.to_csv(os.path.join(os.path.pardir, "data/combined_data.csv"))

CPU times: total: 5min 13s
Wall time: 5min 39s


## EDA using pandas

In [25]:
%%time
df = pd.read_csv(os.path.join(os.path.pardir, "data/combined_data.csv"), nrows=10)

CPU times: total: 0 ns
Wall time: 6 ms


dtype('float64')